# This is a simple graph

In [2]:
import tensorflow as tf

In [3]:
a = tf.constant((3,4), name="input_a")
b = tf.constant(2, name="input_b")

In [4]:
c = tf.multiply(a,b, name="mul_c")
d = tf.add(a,b,name="add_d")

In [5]:
e = tf.add(c,d,name="add_e")

In [6]:
sess=tf.Session()
sess.run(e)

array([11, 14], dtype=int32)

# creating writer object for displaying graph

In [7]:
writer = tf.summary.FileWriter('./my_graph',sess.graph)

In [8]:
writer.close()
sess.close()

# Second example: implementing from graph

In [9]:
a =tf. constant([5,3], name = "input_a")

In [10]:
b = tf.reduce_prod(a, name="prod_b")
c = tf.reduce_sum(a,name="sum_c")

In [11]:
d = tf.add(b,c, name="add_d")

In [12]:
shape = tf.shape(a)

test =tf.add(shape, a)

In [13]:
sess = tf.Session()
sess.run(test)

array([7, 5], dtype=int32)

In [14]:
sess.close()

# initialitzing variables

In [15]:
import tensorflow as tf

In [16]:
normal = tf.random_normal([3,3,3], mean= 0.0, stddev= 2.0)

In [17]:
init=tf.initialize_all_variables()
sess=tf.Session()
sess.run(init)

Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [18]:
sess = tf.Session()
sess.run(normal)

array([[[-3.0722010e+00,  2.1409199e+00, -6.4811572e-03],
        [-2.7822800e+00,  1.6969399e+00,  1.3374985e+00],
        [-8.2613349e-01, -1.6888282e-01,  1.4737719e+00]],

       [[-1.0227950e+00, -2.7539084e+00,  6.6289854e-01],
        [-5.1754746e+00,  1.6089920e+00,  3.8452718e+00],
        [-2.8862143e+00, -2.3899910e-01,  1.1735686e+00]],

       [[-7.5967723e-01, -4.0556345e-02,  4.2363205e+00],
        [-5.1367617e-01, -1.1445843e+00, -7.3834634e+00],
        [ 5.6170511e-01,  3.2579861e+00,  1.5599737e+00]]], dtype=float32)

# exploring scopes

In [21]:
with tf.name_scope("Scope_A"):
    a = tf.add(1,2,name="A_add")
    b = tf.multiply(a,3,name="B_mul")
    
with tf.name_scope("Scope_B"):
    c = tf.add(4,5, name="C_add")
    d = tf.multiply(c,6, name="B_mul")
    
e = tf.add(b,d, name= "e_add")

In [25]:

writer=tf.summary.FileWriter('./name_scope_1',graph=tf.get_default_graph())

In [26]:
writer.close()

# Exercise, putting it all together
 

In [165]:
import tensorflow as tf

In [166]:
#excplicitly creating a graph
graph = tf.Graph()

In [167]:
#defining our data flow graph
with graph.as_default():

    with tf.name_scope("global_variables"):
        #variable to keep track of number of runs
        global_step = tf.Variable(0, dtype = tf.int32, trainable = False, name= "global_step")

        #varibale to keep track of sum of output over all runs of model
        total_output = tf.Variable(0, dtype = tf.float32, trainable = False, name = "total_output")
        
    with tf.name_scope("transformation"):
        
        #seperate input layer
        with tf.name_scope("input"):
            #create input placeholder
            a=tf.placeholder(tf.float32, shape=None, name="input_placeholder_a")            
        #seperate middle layer
        with tf.name_scope("intermediate_layer"):
            b = tf.reduce_prod(a, name = "product_b")
            c = tf.reduce_sum(a, name="sum_a")
        
        #final output layer    
        with tf.name_scope("output"):
            output = tf.add(b,c, name = "output")
    
    #update scope
    with tf.name_scope("update"):
        #increment total update
        update_total = total_output.assign_add(output)
        
        #increment the above gloval step
        increment_step = global_step.assign_add(1)
        
    with tf.name_scope("summaries"):
        avg=tf.div(update_total,tf.cast(increment_step,tf.float32),name="average")
        #Creates summaries for output node
        tf.summary.scalar('Output',output)
        tf.summary.scalar('Sum_of_outputs_overtime',update_total)
        tf.summary.scalar('Average_of_outputs_overtime',avg)
        
    with tf.name_scope("global_ops"):
        #Initialization Op
        init=tf.initialize_all_variables()
        #Merge all summaries into one Operation
        merged_summaries=tf.summary.merge_all()

In [168]:
#run graph
sess = tf.Session(graph=graph)
writer = tf.summary.FileWriter('./improved_graph', graph)

In [169]:
#initialize our variables
sess.run(init)

In [170]:
def run_graph(input_tensor):
    feed_dict={a:input_tensor}
    _,step,summary=sess.run([output,increment_step,merged_summaries],feed_dict=feed_dict)
    writer.add_summary(summary,	global_step=step)

In [171]:
## run different initializtaions of the grpah
run_graph([2,3])
run_graph([3,1,3,3])
run_graph([8])
run_graph([1,2,3])
run_graph([11,4])
run_graph([4,1])
run_graph([7,3,1])
run_graph([6,3])
run_graph([0,2])
run_graph([4,5,6])

In [172]:
writer.flush()

In [173]:
writer.close()
sess.close()